In [34]:
#H0：各组比例相等，解释变量对被解释变量无影响
import pandas  as  pd
import numpy as np
import scipy 
import warnings # 忽略所有警告
warnings.filterwarnings("ignore")

## 相关分析

In [32]:
#两个连续变量间是否存在关系

In [13]:
#相关分析 :  两个连续变量间是否存在线性关系
import numpy as np
import scipy 
# 身高数据（单位：cm）
heights =np.array([162, 155, 168, 172, 158, 180, 175, 163, 170, 185, 160, 177, 165, 182, 169])
# 对应体重数据（单位：kg）
weights =np.array([55, 48, 62, 68, 50, 75, 70, 53, 65, 82, 52, 73, 58, 78, 67])
#手动计算 r：衡量两个变量的线性关系	计算偏离均值的乘积，标准化到[-1,1]
r = np.sum((heights-heights.mean()) * (weights-weights.mean()))/np.sqrt(np.sum((heights-heights.mean())**2) * np.sum((weights-weights.mean())**2))
r

#---------------------------公式计算--------------------------------
corr_table = np.corrcoef(heights,weights) #得到一个相关系数矩阵
corr=corr_table[0,1]
print("相关系数",corr,"\n")

#确定假设: H0:总体相关系数 为 0 ，身高与体重无关
#         H1:总体相关系数不 为 0 ，身高与体重有关

#检验r是否显著（非随机）	t越大，相关性越不可能偶然
corr_t=corr * np.sqrt(weights.size-2)/np.sqrt(1-corr**2)
print("相关系数统计量t",corr_t,"\n") #这个值越大说明相关性越强。

#确定小概率
alpha = 0.05

#t统计量的p值判断是否统计显著	p < 0.05 说明相关性真实存在
p=(1-scipy.stats.t.cdf(corr_t,weights.size-2))*2
print("统计量的P值",p)

if p<alpha:
    print("拒绝原假设, 身高体重有显著的相关性")
else:
    print("接受原假设, 身高体重没有显著的相关性")

#-----------------------函数验证--------------------------------------
cor,p_value=scipy.stats.pearsonr(heights,weights)  
alpha = 0.05
print("相关系数",cor)
print("统计量的P值",p_value)


相关系数 0.9906058102195705 

相关系数统计量t 26.118621309002304 

统计量的P值 1.2798651027878805e-12
拒绝原假设, 身高体重有显著的相关性
相关系数 0.9906058102195707
统计量的P值 1.279863082554445e-12


## 列联分析

In [33]:
#卡方检验：两个分类变量间是否存在关系

In [18]:
data = pd.read_excel(r"D:\DA\4.python\3.统计分析\卡方检验与方差分析案例数据.xlsx",sheet_name=3)
data.head()

total_bili = (data.Click=="点击").sum()/data.shape[0] #总的点击数/总人数
print("总体的点击占比",total_bili)

crosstabl = pd.crosstab(data.Sex,data.Click) #列联交叉表 真实频数
crosstabl

crosstabl_ex = crosstabl.copy() #期望频数表
crosstabl_ex.iloc[:,1] = crosstabl.sum(axis=1) * total_bili
crosstabl_ex.iloc[:,0] = crosstabl.sum(axis=1)-crosstabl_ex.iloc[:,1]
crosstabl_ex 

kafazhi=  ((crosstabl-crosstabl_ex)**2/crosstabl_ex).sum().sum()
kafazhi

p = 1-scipy.stats.chi2.cdf(kafazhi,(crosstabl.shape[0]-1) * (crosstabl.shape[1]-1))
print("卡方统计量",kafazhi,"p",p)

alpha = 0.05
if p<alpha:
    print("拒绝原假设, 性别对是否点击有影响")
else:
    print("接受原假设, 性别对是否点击无影响")

总体的点击占比 0.5652173913043478
卡方统计量 1.8651072124756343 p 0.17203651012018484
接受原假设, 性别对是否点击无影响


In [31]:
#函数验证
s,p,df,ex=scipy.stats.chi2_contingency(crosstabl,correction=False)
print("卡方统计量",s,"p", p)
alpha = 0.05
if p<alpha:
    print("拒绝原假设, 性别对是否点击有影响")
else:
    print("接受原假设, 性别对是否点击无影响")

卡方统计量 1.8651072124756327 p 0.172036510120181
接受原假设, 性别对是否点击无影响


## 方差分析

### eg.1(分三列)

In [36]:
#方差分析：检验多个总体均值是否存在显著差异，通过分析数据判断分类型自变量对数值行因变量是否有影响

# H0:各水平均值相等，
import pandas  as  pd
import numpy as np
import scipy 

#某教育机构研究三种教学方法（A、B、C）对学生数学成绩的影响	
data = pd.read_excel(r"D:\DA\4.python\3.统计分析\卡方检验与方差分析案例数据.xlsx",sheet_name=1)
data.head()

#SST = 总差异(每个变量与总体均值的差异平方和)
SST = ((data-data.mean(None))**2).sum().sum()
print("SST",SST)

#SSA=各水平均值与总体均值的差异平方和
SSA = (((data.mean() - data.mean(None))**2)*data.count()).sum()
MSA = SSA/(data.columns.size-1)
print("SSA",SSA , "MSA",MSA)

#SSE=各变量与所在水平的均值差异平方和	 = SST-SSA
SSE = SST-SSA
print(SSE)

MSE = SSE/(data.size-data.columns.size)
print("SSE",SSE,"MSE",SSE/(data.size-data.columns.size))

F = MSA/MSE #组间均方/组内均方
print("F",F,"自由度", data.columns.size-1,data.size-data.columns.size)
P = 1- scipy.stats.f.cdf(F,data.columns.size-1,data.size-data.columns.size)
print("F","P",P)

#小概率 alpha 
alpha =0.05
np.where(P<alpha,"拒绝原假设：结论各组均值有明显差异","接受原假设：结论各各组均值无明显差异")

SST 35479.65079365079
SSA 6416.222222222223 MSA 3208.1111111111113
29063.42857142857
SSE 29063.428571428572 MSE 484.3904761904762
F 6.622985522633583 自由度 2 60
F P 0.0025178235002747407


array('拒绝原假设：结论各组均值有明显差异', dtype='<U18')

In [44]:
#函数验证
statistic,pvalue = scipy.stats.f_oneway(data.A,data.B,data.C)
print(statistic,pvalue)
#小概率 alpha
alpha =0.05
np.where(pvalue<alpha,"拒绝原假设：结论各组均值有明显差异","接受原假设：结论各组均值无明显差异")

6.622985522633584 0.002517823500274781


array('拒绝原假设：结论各组均值有明显差异', dtype='<U17')

In [ ]:
#条件语句
#np.where(条件语句,条件满足时的内容,条件不满足时的内容) #可嵌套

###  eg.2(在一列)

In [70]:
#某教育机构研究三种教学方法（A、B、C）对学生数学成绩的影响	
data = pd.read_excel(r"D:\DA\4.python\3.统计分析\卡方检验与方差分析案例数据.xlsx",sheet_name=4)
data.head()
#SST
SST = np.sum((data.成绩-data.成绩.mean()) **2)
SST

#SSA
SSA = np.sum(((data.groupby("教学方法")["成绩"].mean()-data.成绩.mean())**2) * data.groupby("教学方法")["成绩"].count())
MSA = SSA/(data.教学方法.nunique()-1)
print(SSA,MSA)

SSE = SST-SSA
#SSE=data.apply(lambda x:(x[1]-data.成绩[data.教学方法==x[0]].mean())**2,axis=1).sum()
MSE=SSE/(data.成绩.size-data.教学方法.nunique())

F = MSA/MSE
F_p= 1-scipy.stats.f.cdf(F,data.教学方法.nunique()-1,data.成绩.size-data.教学方法.nunique())
print(F,F_p)
np.where(F_p<0.05,"拒绝原假设: 结论是 教学方法对成绩有影响","接受原假设: 结论是 教学方法对成绩无影响")

6416.222222222223 3208.1111111111113
6.622985522633584 0.0025178235002747407


array('拒绝原假设: 结论是 教学方法对成绩有影响', dtype='<U21')

### eg.3(函数)

In [86]:
scipy.stats.f_oneway(data.成绩[data.教学方法=="A"],data.成绩[data.教学方法=="B"],data.成绩[data.教学方法=="C"])

F_onewayResult(statistic=np.float64(6.622985522633584), pvalue=np.float64(0.002517823500274781))